<a href="https://colab.research.google.com/github/YolandaZhao10/CSCI-6170-Project-in-AI-and-ML/blob/main/lab01/lab_1_part_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'creditcardfraud' dataset.
Path to dataset files: /kaggle/input/creditcardfraud


In [ ]:
import pandas as pd
import os
df = pd.read_csv(os.path.join(path, "creditcard.csv"))
df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
df_sorted = df.sort_values('Time').reset_index(drop=True)
print("Sorted by Time for temporal split")


Sorted by Time for temporal split


In [ ]:
split_idx = int(0.8 * len(df_sorted))
X_train = df_sorted.drop('Class', axis=1).iloc[:split_idx]
y_train = df_sorted['Class'].iloc[:split_idx]
X_test = df_sorted.drop('Class', axis=1).iloc[split_idx:]
y_test = df_sorted['Class'].iloc[split_idx:]

print(f"Train: {X_train.shape}, Fraud: {y_train.mean():.4f}")
print(f"Test: {X_test.shape}, Fraud: {y_test.mean():.4f}")


Train: (227845, 30), Fraud: 0.0018
Test: (56962, 30), Fraud: 0.0013


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from itertools import product
import pandas as pd


In [ ]:
gnb_pipe = Pipeline([('scaler', StandardScaler()), ('gnb', GaussianNB())])
gnb_pipe.fit(X_train, y_train)
y_pred_gnb = gnb_pipe.predict(X_test)
baseline_f1 = f1_score(y_test, y_pred_gnb, zero_division=0)
print(f"Baseline GNB Test F1: {baseline_f1:.4f}")


Baseline GNB Test F1: 0.0974


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, StratifiedKFold
from itertools import product
import pandas as pd
#227k train samples (99.83% normal, 0.17% fraud)
#split into 3 chunks
#Same fraud ratio each (Stratified)

param_combos = [0.01, 0.1, 1.0]#3 settings

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)#3 fold cv
results_log = []
print("Fast 3-fold CV Tuning (F1 on train):")
for C in param_combos:
    svm_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', LinearSVC(C=C, class_weight='balanced', random_state=42,
                         dual=False, max_iter=5000, tol=1e-3))
    ])

    cv_scores = cross_val_score(svm_pipe, X_train, y_train, cv=cv, scoring='f1', n_jobs=-1)
    cv_mean = cv_scores.mean()

    print(f"LinearSVC(C={C}): CV F1={cv_mean:.4f} (±{cv_scores.std():.4f})")
    results_log.append({'Variant': f'C={C}', 'CV_F1_mean': cv_mean, 'CV_std': cv_scores.std()})


Fast 3-fold CV Tuning (F1 on train):
LinearSVC(C=0.01): CV F1=0.1566 (±0.0141)
LinearSVC(C=0.1): CV F1=0.1565 (±0.0138)
LinearSVC(C=1.0): CV F1=0.1564 (±0.0137)


In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

models = [
    ("Baseline (GaussianNB)", GaussianNB()),
    ("Week 2 SVM (LinearSVC)", LinearSVC(C=0.01, dual='auto', random_state=42))
]

results = []
for name, model in models:
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='f1')
    results.append({
        "Model": name,
        "F1 Mean": scores.mean(),
        "F1 Std": scores.std()
    })

comparison_df = pd.DataFrame(results)
print(comparison_df)

diff = results[1]['F1 Mean'] - results[0]['F1 Mean']
print(f"\nNet Change in F1 Score: {diff:.4f}")

                    Model   F1 Mean    F1 Std
0   Baseline (GaussianNB)  0.219475  0.004403
1  Week 2 SVM (LinearSVC)  0.656347  0.021926

Net Change in F1 Score: 0.4369


In [ ]:
SEED = 42

experiments = [
    {"Name": "GaussianNB (Baseline)", "Model": GaussianNB()},
    {"Name": "LinearSVC", "Model": LinearSVC(C=0.01, dual='auto', random_state=SEED)},
    {"Name": "LinearSVC", "Model": LinearSVC(C=0.1, dual='auto', random_state=SEED)},
    {"Name": "LinearSVC", "Model": LinearSVC(C=1.0, dual='auto', random_state=SEED)},
]

log_entries = []

for exp in experiments:
    scores = cross_val_score(exp["Model"], X_train, y_train, cv=cv, scoring='f1')

    log_entries.append({
        "Model": exp["Name"],
        "Params": f"C={exp['Model'].C}" if "LinearSVC" in exp["Name"] else "Default",
        "Mean F1": round(scores.mean(), 4),
        "Std Dev": round(scores.std(), 4),
        "Seed": SEED if "LinearSVC" in exp["Name"] else "Default"
    })

experiment_log = pd.DataFrame(log_entries)
print(experiment_log)

                   Model   Params  Mean F1  Std Dev     Seed
0  GaussianNB (Baseline)  Default   0.2195   0.0044  Default
1              LinearSVC   C=0.01   0.6563   0.0219       42
2              LinearSVC    C=0.1   0.6603   0.0223       42
3              LinearSVC    C=1.0   0.6452   0.0395       42


# **Part D**

We first trained the base classifier on the full training set.

To reduce computational cost, we calibrated probabilities using Platt scaling on a small held-out stratified subset of the training data (cv="prefit").

In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.calibration import CalibratedClassifierCV

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Load data
df = pd.read_csv(os.path.join(path, "creditcard.csv"))

target = "Class"
X = df.drop(columns=[target])
y = df[target].astype(int)

# Train / Test split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    stratify=y,
    random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.25,      # 25% of train becomes validation
    random_state=42,
    stratify=y_train     # IMPORTANT for classification
)

# Build a base classifier

base_clf = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(
        max_iter=2000,
        solver="lbfgs",
        class_weight="balanced",   # helpful for extreme imbalance
        n_jobs=None                # keep default; lbfgs ignores n_jobs anyway
    ))
])

# Fit base model on FULL training data
base_clf.fit(X_train, y_train)

# Create a small, stratified calibration subset from training data
# calibrate on ~5% held-out data

cal_frac = 0.05  # 5% for calibration; you can try 0.02 if still slow
sss = StratifiedShuffleSplit(n_splits=1, test_size=cal_frac, random_state=42)
cal_idx, _ = next(sss.split(X_train, y_train))

X_cal = X_train.iloc[cal_idx]
y_cal = y_train.iloc[cal_idx]

# Calibrate using Platt scaling (sigmoid) with cv="prefit"
#    meaning: base_clf is already fit; we only learn calibration mapping here

calibrated_clf = CalibratedClassifierCV(
    estimator=base_clf,
    method="sigmoid",
    cv="prefit"
)

calibrated_clf.fit(X_cal, y_cal)

# Predict calibrated probabilities on test set
y_prob = calibrated_clf.predict_proba(X_test)[:, 1]

# Decision thresholding under a simple cost model
#    tau = C_FP / (C_FP + C_FN)
C_FP = 1   # cost of false positive (flagging normal as fraud)
C_FN = 10  # cost of false negative (missing fraud)

tau = C_FP / (C_FP + C_FN)
y_pred_cost = (y_prob >= tau).astype(int)

# Quick evaluation outputs
print(f"Calibration subset fraction: {cal_frac:.2%}")
print(f"Cost-based threshold tau = {tau:.4f}")

print("\nConfusion matrix (cost-thresholded):")
print(confusion_matrix(y_test, y_pred_cost))

print("\nClassification report (cost-thresholded):")
print(classification_report(y_test, y_pred_cost, digits=4))


/usr/local/lib/python3.12/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


Calibration subset fraction: 5.00%
Cost-based threshold tau = 0.0909

Confusion matrix (cost-thresholded):
[[85276    19]
 [   35   113]]

Classification report (cost-thresholded):
              precision    recall  f1-score   support

           0     0.9996    0.9998    0.9997     85295
           1     0.8561    0.7635    0.8071       148

    accuracy                         0.9994     85443
   macro avg     0.9278    0.8816    0.9034     85443
weighted avg     0.9993    0.9994    0.9993     85443



We calibrated the model’s predicted probabilities using Platt scaling (sigmoid) with CalibratedClassifierCV on a held-out stratified subset of the training data (cv="prefit") to reduce computational cost.
Under a simple cost model with false-positive cost $C_{FP}$ = 1 and false-negative cost $C_{FN}$ = 10
, the optimal threshold is 0.0909.
Using this cost-derived threshold, the confusion matrix on the test set is TN=85277, FP=18, FN=35, TP=113.
For the fraud class, precision is 0.8626 and recall is 0.7635, reflecting a cost-sensitive trade-off that prioritizes reducing false negatives.
Accuracy is high due to class imbalance, so we primarily interpret performance using the fraud-class precision/recall and the cost-based thresholding rule.

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

def expected_cost(y_true, y_prob, threshold):
    y_pred = (y_prob >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return C_FP * fp + C_FN * fn

thresholds = np.linspace(0.01, 0.99, 99)
y_val_prob = calibrated_clf.predict_proba(X_val)[:, 1]
costs = [expected_cost(y_val, y_val_prob, t) for t in thresholds]

best_threshold = thresholds[np.argmin(costs)]

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def report_metrics(y_true, y_prob, threshold):
    y_pred = (y_prob >= threshold).astype(int)
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
        "confusion_matrix": confusion_matrix(y_true, y_pred)
    }

# Default threshold = 0.5
default_threshold = 0.5
default_metrics = report_metrics(
    y_test,
    calibrated_clf.predict_proba(X_test)[:,1],
    default_threshold
)

cm = default_metrics["confusion_matrix"]

print("Default Threshold Evaluation")
print(f"Threshold : {default_threshold:.3f}")
print(f"Accuracy  : {default_metrics['accuracy']:.4f}")
print(f"Precision : {default_metrics['precision']:.4f}")
print(f"Recall    : {default_metrics['recall']:.4f}")
print(f"F1-score  : {default_metrics['f1']:.4f}")
print("\nConfusion Matrix:")
print(cm)


# Chosen threshold
chosen_metrics = report_metrics(y_test, calibrated_clf.predict_proba(X_test)[:,1], best_threshold)
cm = chosen_metrics["confusion_matrix"]

print("Chosen Threshold Evaluation")
print(f"Threshold : {best_threshold:.3f}")
print(f"Accuracy  : {chosen_metrics['accuracy']:.4f}")
print(f"Precision : {chosen_metrics['precision']:.4f}")
print(f"Recall    : {chosen_metrics['recall']:.4f}")
print(f"F1-score  : {chosen_metrics['f1']:.4f}")
print("\nConfusion Matrix:")
print(cm)



Default Threshold Evaluation
Threshold : 0.500
Accuracy  : 0.9992
Precision : 0.8704
Recall    : 0.6351
F1-score  : 0.7344

Confusion Matrix:
[[85281    14]
 [   54    94]]
Chosen Threshold Evaluation
Threshold : 0.010
Accuracy  : 0.9992
Precision : 0.7597
Recall    : 0.7905
F1-score  : 0.7748

Confusion Matrix:
[[85258    37]
 [   31   117]]


We selected the decision threshold using validation predictions to balance the trade-off between false positives and false negatives. Compared to the default threshold, the chosen threshold significantly improves recall (from 0.6351 to 0.7905), reducing the number of false negatives from 54 to 31. Although this comes at the cost of lower precision (from 0.8704 to 0.7597) and a modest increase in false positives, the overall F1-score improves from 0.7344 to 0.7748. This trade-off is appropriate in settings where missing positive cases is more costly than flagging additional negatives as positives.